In [ ]:
#libraries
library(Seurat)
library(SeuratObject)
library(ggplot2)
library(scCustomize)
library(viridis)
library(clusterProfiler)
library(org.Mm.eg.db) 
library(tidyverse)

In [ ]:
# dataset loading
seurat_object <- readRDS("/path/to/folder/seurat_object.rds")
regulons <- read.delim("/path/to/folder/activities.tsv", 
                       row.names = 1, header = T)

In [ ]:
#creating AUC assay within seurat object
reg <- colnames(regulons)
reg <- stringr::str_remove(reg, "\\.\\.\\.")
colnames(regulons) <- reg
regulons <- t(regulons)
regulons <- as.matrix(regulons)
seurat_object[['AUC']] <- CreateAssayObject(data = regulons)
DefaultAssay(seurat_object) <- 'AUC'
seurat_object <- ScaleData(seurat_object)
seurat_object@assays$AUC@data <- seurat_object@assays$AUC@scale.data

In [ ]:
#finding top active regulons and filtering e.g. top 50
DefaultAssay(seurat_object) <- "AUC"
Idents(seurat_object) <- "annotation"
deg.all <- FindAllMarkers(seurat_object, logfc.threshold = 0.25, test.use = "wilcox", only.pos = TRUE)
deg.ls <- split(deg.all, deg.all$cluster)
gene.ls <- lapply(deg.ls, function(x) head(x$gene, 50)) 
gene.vec <- unique(unlist(gene.ls))  

In [ ]:
# color coding of the clusters
my_colors <- c(
"Endothelial cells" = "#333f71",
"Ameloblasts" = "#ec8b1d",
"(pre)Odontoblasts" = "#3f41ed",
"Dental pulp" = "#aeaff8",
"Glia" = "#57402d",
"Imunne cells" = "#fef5f2",
"Osteocytes" = "#f6d09f",
"Perivascular cells" = "#9e7a58",
"Dental folicle" = "#e94e22"
)
my_colors_odontoblasts <- c(
  "Near-CL area" = "#ef5675",  
  "Pre-Odontoblasts" = "#8c564b",   
  "Early Odontoblasts" = "#58508d",   
  "Late Odontoblasts" = "#f3a583"    
)

In [ ]:
#heatmap vizualization of top active regulons and figure saving 
p <- DoHeatmap(seurat_object, features = gene.vec, assay = "AUC", raster = FALSE, 
               group.by = "annotation", group.colors = my_colors) +  
  scale_fill_viridis_c(option = "plasma") +
  theme(
    axis.text.y = element_text(size = 18),  
    axis.text.x = element_blank(), 
    legend.title = element_text(size = 12),  
    legend.text = element_text(size = 12), 
    legend.key.size = unit(1.5, "cm"),  
    legend.position = "right"  
  ) +
  guides(colour = FALSE)  
output_dir <- "/path/to/folder/" 
ggsave(
  filename = file.path(output_dir, paste0("regulons_activity.svg"))
  plot = p, 
  width = 20, 
  height = 60,
  limitsize = FALSE
)

In [ ]:
# umap vizualization of the regulon activity (e.g. Gsc)
DefaultAssay(seurat_object) <- "AUC"
p3 <- FeaturePlot_scCustom(
  seurat_object,
  features = c('Gsc'),
  pt.size = 2.5,
  alpha_exp = 0.75
) +
  scale_color_viridis(option = "plasma") + 
  theme(
    axis.line = element_blank(),
    axis.text = element_blank(),
    axis.ticks = element_blank(),
    axis.title = element_blank()
  )
ggsave(
  filename = file.path(output_dir, paste0("Gsc_regulon_activity_feature_plot.svg")), 
  plot = p3, 
  width = 8, 
  height = 5, 
  limitsize = FALSE
)

In [ ]:
# heatmap vizualization of the expression levels of individual regulon markers of the e.g. Gsc regulon and figure saving
regulon_lines <- readLines("/path/to/folder/genes.txt")
regulons <- list()
i <- 1
while (i < length(regulon_lines)) {
  line <- trimws(regulon_lines[i])
    if (line != "") {
    regulon_name <- line
    gene_line <- trimws(regulon_lines[i + 1])
    genes <- unlist(strsplit(gene_line, ",\\s*"))
    regulons[[regulon_name]] <- genes
    i <- i + 2
  } else {
    i <- i + 1
  }
}

selected_regulon <- "Gsc(+)"
genes <- regulons[[selected_regulon]]

DefaultAssay(seurat_object) <- "RNA"
p2 <- DoHeatmap(seurat_object, features = genes, assay = "RNA", 
               group.by = "annotation", group.colors = my_colors, label = F, group.bar.height = 0.01) +  # Set the grouping to 'annotation' column
  scale_fill_viridis_c(option = "plasma") +
  theme(
    axis.text.y = element_text(size = 8),  
    axis.text.x = element_blank(),  
    legend.title = element_text(size = 7.5),  
    legend.text = element_text(size = 7.5),  
    legend.key.size = unit(0.5, "cm"), 
    legend.position = "right"
  ) +
  guides(colour = FALSE)  
ggsave("/path/to/folder/expression_of_genes_of_regulon_of_interest.svg", plot = p2, width = 12.4, height = 17.59, units = "in")

In [ ]:
# ORA analysis of genes of Gsc regulon, plotting and figure saving
library(clusterProfiler)
library(org.Mm.eg.db) 
gene_ids <- bitr(genes, fromType = "SYMBOL",
                 toType = "ENTREZID",
                 OrgDb = org.Mm.eg.db)
ego_gsc <- enrichGO(gene       = gene_ids$ENTREZID,
                OrgDb         = org.Mm.eg.db,
                keyType       = "ENTREZID",
                ont           = "BP",
                pAdjustMethod = "BH",
                qvalueCutoff  = 0.05,
                readable      = TRUE)
p_gsc <- dotplot(ego_gsc, showCategory=23) +
  scale_fill_viridis_c(option = "plasma") +
  ggtitle("GO analysis of the Gsc regulon members")
ggsave("/path/to/folder/go_analysis_of_gsc_regulon.svg", plot = p_gsc, width = 8.27, height = 11.69, units = "in")

In [ ]:
sessionInfo()
R version 4.4.1 (2024-06-14)
Platform: x86_64-pc-linux-gnu
Running under: Ubuntu 24.04 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.26.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C               LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8     LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8    LC_PAPER=en_US.UTF-8      
 [8] LC_NAME=C                  LC_ADDRESS=C               LC_TELEPHONE=C             LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Prague
tzcode source: system (glibc)

attached base packages:
[1] stats4    grid      stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lattice_0.22-6         SeuratData_0.2.2.9001  rmdformats_1.0.4       loomR_0.2.1.9000       R6_2.5.1               hdf5r_1.3.11           SeuratDisk_0.0.0.9021  ggupset_0.4.1          scCustomize_2.1.2     
[10] enrichplot_1.22.0      scico_1.5.0            org.Mm.eg.db_3.18.0    AnnotationDbi_1.64.1   IRanges_2.36.0         S4Vectors_0.40.2       Biobase_2.62.0         BiocGenerics_0.48.1    clusterProfiler_4.10.1
[19] SoupX_1.6.2            dtplyr_1.3.1           dbplyr_2.5.0           dplyr_1.1.4            extrafont_0.19         gridExtra_2.3          Nebulosa_1.12.1        patchwork_1.2.0        tibble_3.2.1          
[28] gprofiler2_0.2.3       magrittr_2.0.3         harmony_1.2.0          Rcpp_1.0.13            viridis_0.6.5          viridisLite_0.4.2      ggplot2_3.4.4          Matrix_1.6-5           tidyr_1.3.1           
[37] data.table_1.15.4      here_1.0.1             Seurat_5.2.1           SeuratObject_5.0.2     sp_2.1-4              

loaded via a namespace (and not attached):
  [1] R.methodsS3_1.8.2           urlchecker_1.0.1            goftest_1.2-3               Biostrings_2.70.3           HDF5Array_1.30.1            vctrs_0.6.5                 spatstat.random_3.3-1      
  [8] digest_0.6.36               png_0.1-8                   shape_1.4.6.1               ggrepel_0.9.5               deldir_2.0-4                parallelly_1.38.0           MASS_7.3-60.2              
 [15] reshape2_1.4.4              httpuv_1.6.15               qvalue_2.34.0               withr_3.0.2                 ggrastr_1.0.2               xfun_0.46                   ggfun_0.1.5                
 [22] ellipsis_0.3.2              survival_3.6-4              memoise_2.0.1               ggbeeswarm_0.7.2            janitor_2.2.0               gson_0.1.0                  profvis_0.3.8              
 [29] tidytree_0.4.6              zoo_1.8-12                  GlobalOptions_0.1.2         pbapply_1.7-2               R.oo_1.26.0                 rematch2_2.1.2              KEGGREST_1.42.0            
 [36] promises_1.3.0              httr_1.4.7                  globals_0.16.3              fitdistrplus_1.2-1          rhdf5filters_1.14.1         rhdf5_2.46.1                rstudioapi_0.16.0          
 [43] miniUI_0.1.1.1              generics_0.1.3              DOSE_3.28.2                 zlibbioc_1.48.2             ggraph_2.2.1                polyclip_1.10-7             GenomeInfoDbData_1.2.11    
 [50] SparseArray_1.2.4           xtable_1.8-4                stringr_1.5.1               pracma_2.4.4                evaluate_0.24.0             S4Arrays_1.2.1              GenomicRanges_1.54.1       
 [57] bookdown_0.40               irlba_2.3.5.1               colorspace_2.1-1            ROCR_1.0-11                 reticulate_1.40.0           spatstat.data_3.1-2         lmtest_0.9-40              
 [64] snakecase_0.11.1            later_1.3.2                 ggtree_3.10.1               spatstat.geom_3.3-2         future.apply_1.11.2         scuttle_1.12.0              scattermore_1.2            
 [71] shadowtext_0.1.4            cowplot_1.1.3               matrixStats_1.3.0           RcppAnnoy_0.0.22            pillar_1.9.0                nlme_3.1-164                compiler_4.4.1             
 [78] beachmat_2.18.1             RSpectra_0.16-2             stringi_1.8.4               devtools_2.4.5              tensor_1.5                  SummarizedExperiment_1.32.0 lubridate_1.9.3            
 [85] plyr_1.8.9                  crayon_1.5.3                abind_1.4-5                 gridGraphics_0.5-1          locfit_1.5-9.10             graphlayouts_1.1.1          bit_4.0.5                  
 [92] fastmatch_1.1-4             codetools_0.2-20            paletteer_1.6.0             plotly_4.10.4               mime_0.12                   splines_4.4.1               circlize_0.4.16            
 [99] fastDummies_1.7.3           sparseMatrixStats_1.14.0    HDO.db_0.99.1               Rttf2pt1_1.3.12             knitr_1.48                  blob_1.2.4                  utf8_1.2.4                 
[106] fs_1.6.4                    listenv_0.9.1               DelayedMatrixStats_1.24.0   pkgbuild_1.4.4              ggplotify_0.1.2             statmod_1.5.0               pheatmap_1.0.12            
[113] tweenr_2.0.3                pkgconfig_2.0.3             tools_4.4.1                 cachem_1.1.0                RSQLite_2.3.7               DBI_1.2.3                   fastmap_1.2.0              
[120] rmarkdown_2.27              scales_1.3.0                usethis_3.0.0               ica_1.0-3                   BiocManager_1.30.25         ggprism_1.0.5               dotCall64_1.1-1            
[127] RANN_2.6.1                  farver_2.1.2                tidygraph_1.3.1             scatterpie_0.2.3            yaml_2.3.10                 MatrixGenerics_1.14.0       cli_3.6.3                  
[134] purrr_1.0.2                 lifecycle_1.0.4             uwot_0.2.2                  mvtnorm_1.2-5               presto_1.0.0                sessioninfo_1.2.2           DropletUtils_1.22.0        
[141] BiocParallel_1.36.0         timechange_0.3.0            gtable_0.3.5                ggridges_0.5.6              progressr_0.14.0            parallel_4.4.1              ape_5.8                    
[148] limma_3.58.1                jsonlite_1.8.8              edgeR_4.0.16                RcppHNSW_0.6.0              bitops_1.0-8                bit64_4.0.5                 Rtsne_0.17                 
[155] yulab.utils_0.1.7           spatstat.utils_3.0-5        GOSemSim_2.28.1             dqrng_0.4.1                 spatstat.univar_3.0-0       R.utils_2.12.3              lazyeval_0.2.2             
[162] shiny_1.9.1                 htmltools_0.5.8.1           GO.db_3.18.0                sctransform_0.4.1           rappdirs_0.3.3              glue_1.7.0                  spam_2.10-0                
[169] XVector_0.42.0              RCurl_1.98-1.16             decoupleR_2.8.0             rprojroot_2.0.4             treeio_1.26.0               mclust_6.1.1                ks_1.14.2                  
[176] igraph_2.0.3                extrafontdb_1.0             SingleCellExperiment_1.24.0 labeling_0.4.3              forcats_1.0.0               cluster_2.1.6               pkgload_1.4.0              
[183] Rhdf5lib_1.24.2             aplot_0.2.3                 GenomeInfoDb_1.38.8         DelayedArray_0.28.0         tidyselect_1.2.1            vipor_0.4.7                 ggforce_0.4.2              
[190] future_1.34.0               munsell_0.5.1               KernSmooth_2.23-24          pochi_0.1.0                 htmlwidgets_1.6.4           fgsea_1.28.0                RColorBrewer_1.1-3         
[197] rlang_1.1.4                 spatstat.sparse_3.1-0       spatstat.explore_3.3-1      remotes_2.5.0               fansi_1.0.6                 beeswarm_0.4.0    